# Imports

In [1]:
import pandas as pd
from selenium.webdriver.chrome.options import Options
import os
import re

# Import your custom modules
from database import SQLiteDB
from scraper import scrape_valmont
from processing import normalize_text, clean_capacity, find_best_match, category_groups
from selenium import webdriver


# Part 1: DB

In [2]:
# Initialize the database
db = SQLiteDB("products.db")


In [3]:
# Initialize the database
db = SQLiteDB("products.db")

In [4]:
# Create table

db = SQLiteDB()

db.create_table("""
CREATE TABLE IF NOT EXISTS demo (
    id INTEGER PRIMARY KEY,
    name TEXT,
    value INTEGER
)
""")


In [5]:
# Create operation (i.e. insert)

db.insert(
    "INSERT INTO demo (name, value) VALUES (?, ?)",
    ("test", 123)
)


In [6]:
# Read operation

db.fetch("SELECT * FROM demo")

[(1, 'test', 123)]

In [7]:
# Update operation

db.update(
    "UPDATE demo SET value = ? WHERE name = ?",
    (999, "test")
)

In [8]:
# Delete operation

db.delete(
    "DELETE FROM demo WHERE name = ?",
    ("test",)
)

# Part 2: Scrapping

In [9]:
options = Options()
options.add_argument("--window-size=1920,1080")

# options.add_argument("--headless=new") # Optional

urls = [
    "https://www.profumeriaideale.com/brand/190-valmont",
    "https://www.profumeriaideale.com/brand/190-valmont?page=2",
    "https://www.profumeriaideale.com/brand/190-valmont?page=3",
    "https://www.profumeriaideale.com/brand/190-valmont?page=4"
]

# Run the scraper
scraped_results = scrape_valmont(urls, options)

# Save the raw data to Excel as requested by the assignment
df_raw = pd.DataFrame(scraped_results)
df_raw.to_excel("valmont_raw_scraped.xlsx", index=False)
print(f"Successfully scraped {len(df_raw)} products.")

Scraping Listing Page: https://www.profumeriaideale.com/brand/190-valmont
Scraping Listing Page: https://www.profumeriaideale.com/brand/190-valmont?page=2
Scraping Listing Page: https://www.profumeriaideale.com/brand/190-valmont?page=3
Scraping Listing Page: https://www.profumeriaideale.com/brand/190-valmont?page=4
Successfully scraped 90 products.


# Part 3: Cleaning and Inserting Data

In [10]:
# 1. Create the production table
db.create_table("""
CREATE TABLE IF NOT EXISTS valmont_offers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name TEXT,
    capacity TEXT,
    price REAL,
    product_url TEXT,
    match_key TEXT
)
""")

# 2. Clean the scraped data
df_scraped_clean = df_raw.copy()

# Use apply to clean capacity and normalize product names
df_scraped_clean['capacity'] = df_scraped_clean.apply(
    lambda x: clean_capacity(x['capacity'], x['product_name']), axis=1
)
df_scraped_clean['match_name'] = df_scraped_clean['product_name'].apply(normalize_text)

# Create the final match_key
df_scraped_clean['match_key'] = (
    df_scraped_clean['match_name'] + " " + df_scraped_clean['capacity']
).str.strip()

# 3. Insert into DB 
db.delete("DELETE FROM valmont_offers", ())

for _, row in df_scraped_clean.iterrows():
    # Define the SQL query
    query = """
    INSERT INTO valmont_offers 
    (product_name, capacity, price, product_url, match_key) 
    VALUES (?, ?, ?, ?, ?)
    """
    
    # Define the data as a tuple
    params = (
        row['product_name'], 
        row['capacity'], 
        row['price'], 
        row['product_url'], 
        row['match_key']
    )
    
    # Execute insertion
    db.insert(query, params)

print(f"Cleaned and inserted {len(df_scraped_clean)} records into the database.")

Cleaned and inserted 90 records into the database.


# Part 4: Entity Matching

In [11]:
df_refs = pd.read_excel("../Data_2025/refs.xlsx")
df_refs['match_cap'] = df_refs['capacity'].apply(clean_capacity)
df_refs['match_name'] = df_refs['product_name'].apply(normalize_text)
df_refs['match_key'] = (df_refs['match_name'] + " " + df_refs['match_cap']).str.strip()

# 2. Fetch data from DB
scraped_data = pd.DataFrame(
    db.fetch("SELECT * FROM valmont_offers"), 
    columns=["id", "product_name", "capacity", "price", "product_url", "match_key"]
)
# 3. Matching Logic
ref_groups = df_refs.groupby('match_cap')
results = []

for _, row in scraped_data.iterrows():
    s_cap = row['capacity']
    best_match, score = None, 0
    ref_info = None

    if s_cap != "N/A" and s_cap in ref_groups.groups:
        relevant_refs = ref_groups.get_group(s_cap)
        best_match, score = find_best_match(row['match_key'], relevant_refs['match_key'].tolist())
        
        if best_match:
            ref_info = relevant_refs[relevant_refs['match_key'] == best_match].iloc[0]

    # Category Guard (Strict check to prevent mismatching Eye vs Face creams)
    if ref_info is not None:
        scraped_lower = row['product_name'].lower()
        match_lower = ref_info['product_name'].lower()
        for cat_name, synonyms in category_groups.items():
            in_scrape = any(re.search(rf'\b{s}\b', scraped_lower) for s in synonyms)
            in_match = any(re.search(rf'\b{s}\b', match_lower) for s in synonyms)
            if in_scrape != in_match:
                ref_info = None
                score = 0
                break

    # 4. Save result with ALL columns from original requirement
    results.append({
        'ID': row['id'], 
        'scraped_name': row['product_name'], 
        'scraped_capacity': row['capacity'],    
        'matched_ref': ref_info['product_name'] if ref_info is not None else None, 
        'ref_capacity': ref_info['capacity'] if ref_info is not None else None,   
        'SKU': ref_info['SKU'] if ref_info is not None else None, 
        'product_url': row['product_url'],      
        'score': score,
        'cap_debug': s_cap
    })

# Final Export
df_matches = pd.DataFrame(results)
match_rate = (df_matches['SKU'].notnull().sum() / len(df_matches)) * 100
print(f"Final Match Rate: {match_rate:.2f}%")

df_matches.to_excel("matches.xlsx", index=False)
display(df_matches.head())

Final Match Rate: 91.11%


,ID,scraped_name,scraped_capacity,matched_ref,ref_capacity,SKU,product_url,score,cap_debug
0,271,Valmont L'elixir Glaciers Serum Majestueux Vos...,15ml,SERUM MAJESTUEUX VOS YEUX,15ml,900503.0,https://www.profumeriaideale.com/occhi/30582-v...,0.697552,15ml
1,272,Valmont Storie Veneziane Alessandrite I'Extrai...,100ml,Alessandrite I,100ml,801001.0,https://www.profumeriaideale.com/profumi-donna...,0.743182,100ml
2,273,Valmont L'elixir Des Glaciers Teint Précieux F...,N/A,None,None,NaN,https://www.profumeriaideale.com/viso/30232-23...,0.000000,N/A
3,274,Valmont Hand Moistrurizing Cream 100,100ml,HAND NUTRITIVE TREATMENT,100ml,703022.0,https://www.profumeriaideale.com/trattamento-c...,0.967069,100ml
4,275,Valmont Fluid Falls Latte Detergente Struccant...,150ml,FLUID FALLS,150ml,705040.0,https://www.profumeriaideale.com/trattamento-v...,1.000000,150ml
